In [ ]:
!pip install transformers datasets tokenizers seqeval accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


In [ ]:
import torch
import datasets
from transformers import AutoModel, AutoTokenizer
import numpy as np
from transformers import BertTokenizerFast
from transformers import BertTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

covid_data = datasets.load_dataset("truongpdd/NER-covid-vietnamese")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5027 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
covid_data.shape

{'train': (5027, 2), 'validation': (2000, 2)}

In [ ]:
print(covid_data['train'][0])

{'tokens': ['Đồng', 'thời', ',', 'bệnh', 'viện', 'tiếp', 'tục', 'thực', 'hiện', 'các', 'biện', 'pháp', 'phòng', 'chống', 'dịch', 'bệnh', 'COVID', '-', '19', 'theo', 'hướng', 'dẫn', 'của', 'Bộ', 'Y', 'tế', '.'], 'ner_tags': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORGANIZATION', 'I-ORGANIZATION', 'I-ORGANIZATION', 'O']}


In [ ]:
all_ner_tags_train = [ner_tags for example in covid_data['train'] for ner_tags in example['ner_tags']]

ner_tags_set = np.unique(all_ner_tags_train)
print('Number of tags: {} '.format(len(ner_tags_set)))
for e in ner_tags_set:
    print(e)

Number of tags: 21 
B-AGE
B-DATE
B-GENDER
B-JOB
B-LOCATION
B-NAME
B-ORGANIZATION
B-PATIENT_ID
B-SYMPTOM_AND_DISEASE
B-TRANSPORTATION
I-AGE
I-DATE
I-GENDER
I-JOB
I-LOCATION
I-NAME
I-ORGANIZATION
I-PATIENT_ID
I-SYMPTOM_AND_DISEASE
I-TRANSPORTATION
O


In [ ]:
ner_tags_mapping = {
    'B-AGE': 0,
    'B-DATE': 1,
    'B-GENDER': 2,
    'B-JOB': 3,
    'B-LOCATION': 4,
    'B-NAME': 5,
    'B-ORGANIZATION': 6,
    'B-PATIENT_ID': 7,
    'B-SYMPTOM_AND_DISEASE': 8,
    'B-TRANSPORTATION': 9,
    'I-AGE': 10,
    'I-DATE': 11,
    'I-GENDER': 12,
    'I-JOB': 13,
    'I-LOCATION': 14,
    'I-NAME': 15,
    'I-ORGANIZATION': 16,
    'I-PATIENT_ID': 17,
    'I-SYMPTOM_AND_DISEASE': 18,
    'I-TRANSPORTATION': 19,
    'O': 20,
}

In [ ]:
def map_ner_tags_to_numbers(example):
    example['ner_tags'] = [ner_tags_mapping[tag] for tag in example['ner_tags']]
    return example

# Apply the mapping to the training set
covid_data['train'] = covid_data['train'].map(map_ner_tags_to_numbers)

# Apply the mapping to the validation set
covid_data['validation'] = covid_data['validation'].map(map_ner_tags_to_numbers)

Map:   0%|          | 0/5027 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
print(covid_data['train'][0])

{'tokens': ['Đồng', 'thời', ',', 'bệnh', 'viện', 'tiếp', 'tục', 'thực', 'hiện', 'các', 'biện', 'pháp', 'phòng', 'chống', 'dịch', 'bệnh', 'COVID', '-', '19', 'theo', 'hướng', 'dẫn', 'của', 'Bộ', 'Y', 'tế', '.'], 'ner_tags': [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 6, 16, 16, 20]}


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
example_text = covid_data['train'][0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokenized_input.word_ids())
print(tokenized_input)
print(tokens)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, None]
{'input_ids': [101, 11326, 13105, 117, 29789, 13190, 16879, 18847, 11663, 12629, 10788, 11338, 14058, 14233, 19459, 19229, 29789, 10348, 41194, 118, 10284, 12986, 17282, 10214, 10500, 11117, 167, 10238, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'đong', 'thoi', ',', 'benh', 'vien', 'tiep', 'tuc', 'thuc', 'hien', 'cac', 'bien', 'phap', 'phong', 'chong', 'dich', 'benh', 'co', '##vid', '-', '19', 'theo', 'huong', 'dan', 'cua', 'bo', 'y', 'te', '.', '[SEP]']


In [ ]:
len(example_text['ner_tags']), len(tokenized_input["input_ids"])
# (9, 11)

(27, 30)

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
q = tokenize_and_align_labels(covid_data['train'][4:5])
print(q)

{'input_ids': [[101, 107, 29789, 12383, 45239, 107, 10222, 19459, 10106, 107, 29789, 12383, 43158, 107, 117, 12439, 22263, 117, 10554, 11117, 167, 10238, 30433, 12383, 20490, 49422, 11123, 23016, 10455, 120, 128, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 20, 20, 20, 7, 20, 20, 20, 20, 20, 20, 20, 7, 20, 20, 0, 20, 20, 20, 6, 16, 16, 20, 20, 20, 20, 20, 20, 1, 1, 1, 20, -100]]}


In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
"_______________________________________ 20
benh____________________________________ 20
nhan____________________________________ 20
523_____________________________________ 7
"_______________________________________ 20
va______________________________________ 20
chong___________________________________ 20
la______________________________________ 20
"_______________________________________ 20
benh____________________________________ 20
nhan____________________________________ 20
522_____________________________________ 7
"_______________________________________ 20
,_______________________________________ 20
67______________________________________ 0
tuoi____________________________________ 20
,_______________________________________ 20
đuoc____________________________________ 20
bo______________________________________ 6
y_______________________________________ 16
te______________________________________ 16
ghi_______________________________

In [ ]:
tokenized_datasets = covid_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5027 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:

model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=21)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=3,
weight_decay=0.01,
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = datasets.load_metric("seqeval")

<ipython-input-21-b144b02b338d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
example = covid_data['train'][0]
print(example)

{'tokens': ['Đồng', 'thời', ',', 'bệnh', 'viện', 'tiếp', 'tục', 'thực', 'hiện', 'các', 'biện', 'pháp', 'phòng', 'chống', 'dịch', 'bệnh', 'COVID', '-', '19', 'theo', 'hướng', 'dẫn', 'của', 'Bộ', 'Y', 'tế', '.'], 'ner_tags': [20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 6, 16, 16, 20]}


label_list = covid_data["train"].features["ner_tags"].feature.names

label_list

In [ ]:


labels = [ner_tags_set[i] for i in example["ner_tags"]]

metric.compute(predictions=[labels], references=[labels])

{'ORGANIZATION': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [ner_tags_set[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [ner_tags_set[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [ ]:
trainer = Trainer(
    model,
    args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.132916,0.881293,0.903907,0.892457,0.963121
2,0.204300,0.107719,0.920815,0.925871,0.923336,0.972451
3,0.204300,0.101092,0.920402,0.938965,0.929591,0.974680


TrainOutput(global_step=945, training_loss=0.1335206147854921, metrics={'train_runtime': 300.5415, 'train_samples_per_second': 50.179, 'train_steps_per_second': 3.144, 'total_flos': 565220719110384.0, 'train_loss': 0.1335206147854921, 'epoch': 3.0})

In [ ]:
model.save_pretrained("ner_model")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(ner_tags_set)
}
label2id = {
    label: str(i) for i,label in enumerate(ner_tags_set)
}

In [ ]:
import json

In [ ]:
config = json.load(open("ner_model/config.json"))

In [ ]:
config["id2label"] = id2label
config["label2id"] = label2id

In [ ]:
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("ner_model")

In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("ner", model=model_fine_tuned, tokenizer=tokenizer)


example = "Đồng thời, bệnh viện tiếp tục thực hiện các biện pháp phòng chống dịch bệnh COVID-19 theo hướng dẫn của Bộ Y tế."

ner_results = nlp(example)

print(ner_results)

[{'entity': 'B-ORGANIZATION', 'score': 0.99143654, 'index': 25, 'word': 'bo', 'start': 104, 'end': 106}, {'entity': 'I-ORGANIZATION', 'score': 0.99496895, 'index': 26, 'word': 'y', 'start': 107, 'end': 108}, {'entity': 'I-ORGANIZATION', 'score': 0.9946439, 'index': 27, 'word': 'te', 'start': 109, 'end': 111}]
